In [100]:
import numpy as np
import pandas as pd
import collections

In [260]:
def read_data(path):
    '''
    Input: 
        path: str
              data path
    Output:
        X: list, shape=(n,)
           list of messages
        Y: ndarray, shape=(n,)
           labels
    '''
    data = pd.read_csv(path, sep='\t', header=None)
    
    X = data[1].str.lower().tolist()
    Y = (data[0] == 'spam').to_numpy() * 1
    
    return X, Y

In [261]:
class NB:
    def __init__(self):
        '''
        Setting up parameters
        '''
        self.phi = None
        self.phiy1 = None
        self.phiy0 = None
        self.vocabulary = None
        
    def make_vocabulary(self, X):
        '''
        Input: 
            X: list, shape(n,)
               list of messages
        Output: 
            vocabulary: dictionary, shape(d,2)
                        vocabulary of data
        '''
        n = len(X_train)
        dict_pos = 0
        vocabulary_word_count = {}
        vocabulary = {}

        for message in X:
            message_word_count = collections.Counter(message.split(' '))
            for word in message_word_count:
                if word in vocabulary_word_count:
                    vocabulary_word_count[word] += 1
                else:
                    vocabulary_word_count[word] = 1

        for word in vocabulary_word_count:
            if vocabulary_word_count[word] >= 5:
                vocabulary[word] = dict_pos
                dict_pos += 1
    
        return dict(sorted(vocabulary.items(), key = lambda x: x[1]))

    def to_vector(self, X, vocabulary):
        '''
        Input: 
            X: list, shape(n,)
               list of messages
            vocabulary: dictionary, shape(d,2)
                        vocabulary of data
        Output:
            X_transformed: ndarray, shape(n,d)
                           messages represented in vector form
        '''
        n = len(X)
        d = len(vocabulary)
        X_transformed = np.zeros((n,d))

        for i, message in enumerate(X):
            word_dict = collections.Counter(message.split(' '))
            for word in word_dict:
                if word in vocabulary:
                    X_transformed[i, vocabulary[word]] = 1

        return X_transformed
    
    def fit(self, X, Y):
        '''
        Input: 
            X: list, shape(n,)
               list of messages
            Y: ndarray, shape(n,)
               labels
        Output: self
        '''
        
        vocabulary = self.make_vocabulary(X)
        X_transformed = self.to_vector(X, vocabulary)
        
        n, d = X_transformed.shape
        
        spam_count = np.sum(Y)
        
        self.phi = spam_count / n
        self.phiy1 = (1 + np.sum(X_transformed[Y == 1], axis=0)) / (2 + spam_count)
        self.phiy0 = (1 + np.sum(X_transformed[Y == 0], axis=0)) / (2 + n - spam_count)
        self.vocabulary = vocabulary
        
        return self
    
    def predict(self, X, threshold):
        '''
        Input:
            X: list, shape(n,)
               list of messages
            threshold: float, 0 <= threshold <= 1
        Output:
            Y: ndarray, shape(n,)
               prediction, p(y=1|x)
        '''
        if type(X) != type([]):
            X_transformed = to_vector([X], self.vocabulary)
        else:
            X_transformed = to_vector(X, self.vocabulary)
            
        n, d = X_transformed.shape
        
    
        p_x_given_y1_matrix = np.outer(np.ones(n), nb.phiy1)
        p_x_given_y0_matrix = np.outer(np.ones(n), nb.phiy0)
        
        p_x_given_y1_matrix[X_transformed == 0] = (1 - np.outer(np.ones(n), nb.phiy1))[X_transformed == 0]
        p_x_given_y0_matrix[X_transformed == 0] = (1 - np.outer(np.ones(n), nb.phiy0))[X_transformed == 0]
        
        p_x_and_y1 = np.sum(np.log(p_x_given_y1_matrix), axis=1) + np.log(nb.phi)
        p_x_and_y0 = np.sum(np.log(p_x_given_y0_matrix), axis=1) + np.log(1 - nb.phi)
        
        px = np.exp(p_x_and_y1) + np.exp(p_x_and_y0)
        
        Y = np.exp(p_x_and_y1) / px
    
        spam_index = np.array(Y > threshold)
        Y[spam_index] = 1
        Y[~spam_index] = 0
        
        return Y
    
    def evaluate(self, Y_predict, Y_true):
        '''
        Input: 
            Y_predict: ndarray, shape(n,)
            Y_true: ndarray, shape(n,)
        Output:
            score: float, 0 <= score <= 1
        '''
        return np.sum(Y_predict == Y_true) / len(Y_true)
    
    def top_spam_words(self, k):
        '''
        Input:
            k: int, 0 <= k
        Output:
            Top k spam words: list
        '''
        arg_spam_word_sorted = np.argsort(np.log(nb.phiy1/nb.phiy0))[::-1]
        inv_vocabulary = {value: key for key, value in nb.vocabulary.items()}
        
        return [inv_vocabulary[arg_spam_word_sorted[i]] for i in range(k)]

In [262]:
X_train, Y_train = read_data('spam_train.tsv')
X_val, Y_val = read_data('spam_val.tsv')
X_test, Y_test = read_data('spam_test.tsv')

In [263]:
nb = NB().fit(X_train, Y_train)

In [264]:
prediction = nb.predict(X_val, .6)
score = nb.evaluate(prediction, Y_val)
print(score)

0.9820466786355476


In [265]:
prediction = nb.predict(X_test, .6)
score = nb.evaluate(prediction, Y_test)
print(score)

0.982078853046595


In [256]:
nb.top_spam_words(10)

['claim',
 'won',
 'prize',
 'urgent!',
 'awarded',
 'tone',
 '£1000',
 'guaranteed',
 '4*',
 '150ppm']